In [6]:
import re
import numpy as np
import pandas as pd

##Imports for gensim, Machine Learning LDA
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

##Imports for NLP
import spacy
import nltk
from nltk.corpus import stopwords

In [7]:
##ReportLab imports, Generating PDFs
from reportlab.pdfgen import canvas
import textwrap
import os
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.pdfbase.pdfmetrics import stringWidth
import errno
from datetime import datetime
import re



In [8]:
# NLTK Stop words
#stop_words = stopwords.words('english')

import nltk
nltk.download('words')
words = set(nltk.corpus.words.words())

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [9]:
email_dataset = pd.read_excel('custom_dataset.xlsx') 
email_dataset_required=email_dataset.drop(['Thread ID', 'Message ID', 'Reply-to','URL Links', 'RFC 822 Message ID', 'Email in PDF', 'Email Addresses'], axis = 1)
email_dataset_bodies=email_dataset_required.drop(['From', 'To', 'Cc','Bcc', 'Subject', 'Date & Time Sent', 'Date & Time Received', 'Labels'], axis = 1)

In [10]:
email_dataset_bodies.head()

,Email Text
0,Here are this week's five links that are worth...
1,"This is my final ""links worth your time"" list ..."
2,Here are this week's five links that are worth...
3,Hey there!\nWe are now 10 full days into the n...
4,Here are this week's five links that are worth...


In [11]:
# Convert to list
data = email_dataset_bodies.values.tolist()

In [12]:
##
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data)) 

In [13]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [14]:
# Define functions for stopwords, bigrams, trigrams and lemmatization

nlp = spacy.load("en_core_web_sm",disable=['parser']) 
 
##Adding additional stopwords
new_stopwords= ['dear', 'thanks','regards', 'hello','hi', 'bye','goodbye', 'say']
for word in new_stopwords:
    nlp.Defaults.stop_words.add(word) 
    nlp.vocab[word].is_stop = True
    
def remove_stopwords_spacy(texts):
    return [[word.text for word in nlp(str(text)) if not word.is_stop] for text in texts]

def remove_stopwords_gensim(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def remove_non_vocab(texts):
    return [[word for word in doc if word in words or not word.isalpha()] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [15]:
# Remove Stop Words
data_words_nostops = remove_stopwords_spacy(data_words)

#remove words that only occur once to make process faster
#all_tokens = sum(data_words_nostops, [])
#tokens_once = set(word for word in set(all_tokens) if all_tokens.count(word) == 1)
#text_no_single_words = [[term for term in words if term not in tokens_once] for words in data_words_nostops]

data_vocab_words=remove_non_vocab(data_words_nostops)

# Form Bigrams
data_words_bigrams = make_bigrams(data_vocab_words)

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [16]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus 
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]



In [17]:
# Build LDA model
coherence_score=[]
perplexity_score=[]
for i in range(1,21):
    
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                               id2word=id2word,
                                               num_topics=i, 
                                               random_state=110,
                                               update_every=1,
                                               chunksize=100,
                                               passes=10,
                                               alpha='auto',
                                               per_word_topics=False,
                                               minimum_probability=0.0)

    perplexity_score.append(lda_model.log_perplexity(corpus))
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    coherence_score.append(coherence_model_lda.get_coherence())

print('Coherence Score', coherence_score)

print('Perplexity Score',perplexity_score)

Coherence Score [0.6363873540754271, 0.5447441159985058, 0.6046194799352524, 0.6364152430394319, 0.46695628900238556, 0.4548813370249462, 0.47862189403385125, 0.5313816494982265, 0.5252433225926189, 0.4965254003381303, 0.4729640101794628, 0.5113232496100382, 0.4613466330239862, 0.5836472878628458, 0.4999093335829733, 0.4792580806961465, 0.4565894422491537, 0.537124199990942, 0.5187633387122961, 0.4765634476653946]
Perplexity Score [-6.391843051554629, -6.339346704094267, -6.344207959469708, -6.3585692348757625, -6.381436030334284, -6.3985415911848715, -6.4144543041366155, -6.415406328372459, -6.433452019336887, -6.433843302085483, -6.463558433320721, -6.4561269695300805, -6.46463870213524, -6.52546596468036, -6.6044618732497025, -6.642596133052682, -6.658619546619261, -6.691374169042024, -6.691466586736132, -6.732172423441828]


In [18]:
f = lambda i: coherence_score[i]

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                               id2word=id2word,
                                               num_topics=max(range(len(coherence_score)), key=f)+1, 
                                               random_state=110,
                                               update_every=1,
                                               chunksize=100,
                                               passes=10,
                                               alpha='auto',
                                               per_word_topics=False,
                                               minimum_probability=0.0)


max(range(len(coherence_score)), key=f)+1
      
      

4

In [19]:
# Marking each email with topic
from itertools import chain
lda_corpus = lda_model[corpus]

all_topics=lda_model.get_document_topics(corpus)

scores = list(chain(*[[score for topic_id,score in topic] \
                      for topic in [doc for doc in lda_corpus]]))
threshold = sum(scores)/len(scores)


all_topics_csr = gensim.matutils.corpus2csc(all_topics)
all_topics_numpy = all_topics_csr.T.toarray()
categorized_docs=[]

for document in all_topics_numpy:
    if max(document) > threshold:
        categorized_docs.append(document.argmax())
    else:
        categorized_docs.append('Unmarked')

email_dataset_bodies = email_dataset_bodies.assign(Topic = categorized_docs)
 

In [20]:
email_dataset_bodies['Topic'].value_counts(dropna=False)

1    106
2      8
Name: Topic, dtype: int64

In [21]:

#Taking top 20 words
topics_with_words=[] #list that contains words belonging to each topic
topic_num_list=[] #topic numbers
for i in range(max(range(len(coherence_score)), key=f)+1):
    tt = lda_model.get_topic_terms(i,20)
    topics_with_words.append([id2word[pair[0]] for pair in tt]) #filling list with each topics word
    topic_num_list.append(i) #filling up the topic number list   
    
#removing stopwords again and removing words that are not in the vocab     
for topic in topics_with_words:
    topic= [word for word in topic if nlp.vocab.has_vector(word)]
    topic= [word for word in topic if not nlp.vocab[word].is_stop]     

topic_dict = dict(zip(topic_num_list, topics_with_words))#buliding a dictionary with list of topic numbers and list of words in those topics

In [22]:
#function that takes in topic words and a category and returns similary between the two
#topic is a list and category should be passsed in as a spacy token
def calculate_similarity(topic_words, category):
    total_sim=0
    string=''
    string = ' '.join([str(elem) for elem in topic_words])## this is the string that contains every word in that topic
    doc= spacy_model(string) ##topic's string is used created a spacy doc
    for token in doc:
        total_sim+= token.similarity(category)
    return total_sim    
    

In [23]:
##Here we firstly have a list of categories one of which will be assigned a category
##Here we calculate similarity of each of the top twenty words of a topic with each category, 
##sum up similarity index for each category for that topic and assign it with the highest one

spacy_model =spacy.load('en_core_web_md')
from spacy.tokenizer import Tokenizer


tokenizer = Tokenizer(spacy_model.vocab)
#list_of_categories =['Personal','Work' ,'Studies', 'Meeting', 'School', 'Bussiness', 'News', 'Subscriptions', 'Marketing', 'Clients',
#                     'Sports', 'Extracurricular', 'Health', 'Travel', 'Schedule', 'Photography',
#                     'Politics', 'Cooking', 'Fashion', 'Social', 'Fitness', 'Research', 'Science', 'Technology', 'Sales',
#                     'Shopping', 'Economy', 'Finance', 'Music', 'Family', 'College', 'Bills', 'Games', 'Design', 'Advertisements',
#                     'Reviews']
#Computer-Science
#Programming-Languages
#Databases
#Cloud computing
#Web-Design
#Software-Development
#Machine-Learning
#Coding-Interviews
#Games
#Productivity
list_of_categories= ['Computer','Programming','Databases','Software','Web','Interviews','Productivity','Games', 'Coding' ]

string_cat=' '.join([str(elem) for elem in list_of_categories])
categories = tokenizer(string_cat)

category_similarity_df = pd.DataFrame(0,index=topic_num_list,columns=list_of_categories)

for category in categories:
    for key, topic_words in topic_dict.items():
        category_similarity_df.loc[key,category.text] = calculate_similarity(topic_words, category)

category_similarity_df

######################################### Things that are left to do:  ################### 

#Run model iteratively until corellation of each topic with a category is above 8 (or has run 10 times)
#Run model iteratively until all topics have a seperate category (or has run 10 times)
#Run model iteratively until we get the best coherence 



#There need to be rules on how much a topic's correlation with a category needs to be for it to be put into it 

#We can do something like after clustering if even one of the topics has similarity with every category less thatn 8 we repeat the 
#clustering and we repeat it till all topics have similarity with one of the categories

#What happens when two topics highly corelated with a single category, that will probably not happen if the list of categories is large

#Try doing that thing where they keep repeating LDA until it has a certain coherence score (you have the saved in OneTab I think)



 

,Computer,Programming,Databases,Software,Web,Interviews,Productivity,Games,Coding
0,4.712211,4.072159,2.955722,3.869267,4.745250,3.448182,3.541660,3.928754,3.662122
1,6.269766,5.665208,4.166120,5.592243,7.146331,4.427041,3.837623,4.871092,4.811071
2,6.765348,4.815424,3.693732,5.223631,6.267658,4.507785,4.136917,5.574654,4.011449
3,4.192877,3.264017,1.907840,3.194795,3.724812,2.868111,2.936002,3.547936,2.669207


In [24]:
import math
topic_category_dict=category_similarity_df.idxmax(axis=1).to_dict() ##dictionary with all topics with their corresponding categories

topic_category_correlation_dict=category_similarity_df.max(axis=1).to_dict() ##dictionary with all topics with their correlation to their categories

thresh=sum(topic_category_correlation_dict.values())/len(topic_category_correlation_dict)   

for key, value in topic_category_correlation_dict.items():
    if topic_category_correlation_dict.get(key)<math.floor(thresh):
        topic_category_dict[key]='Unmarked'

print(topic_category_dict)

email_dataset_bodies['Labels']= email_dataset_bodies['Topic'].map(topic_category_dict)

email_dataset_bodies

print(email_dataset_bodies['Labels'].value_counts(dropna=False))

email_dataset_bodies_remaining=email_dataset_bodies[email_dataset_bodies['Labels']=='Unmarked']
email_dataset_required['Labels'] = email_dataset_bodies['Labels']

#email_dataset_required = email_dataset_required.merge(email_dataset_bodies,on='Labels',how="left")
#email_dataset_bodies_remaining.to_csv('email_dataset_bodies_remaining.csv')




{0: 'Unmarked', 1: 'Web', 2: 'Computer', 3: 'Unmarked'}
Web         106
Computer      8
Name: Labels, dtype: int64


In [25]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.358569226120446

Coherence Score:  0.6364152430394319


In [26]:
email_dataset_required.fillna('', inplace=True)

grouped = email_dataset_required.groupby('Labels')
email_dataset_required.head()

,From,To,Cc,Bcc,Subject,Date & Time Sent,Date & Time Received,Email Text,Labels
0,quincy@freecodecamp.org,malvisbid@gmail.com,,,Learn JavaScript - our free 134-part video cou...,2018-12-13 15:25:58,2021-04-21 16:09:46,Here are this week's five links that are worth...,Web
1,quincy@freecodecamp.org,malvisbid@gmail.com,,,Learn React.js - a free 5-hour course for begi...,2018-12-21 01:09:04,2021-04-21 16:09:44,"This is my final ""links worth your time"" list ...",Web
2,quincy@freecodecamp.org,malvisbid@gmail.com,,,The React Handbook - a massive free guide to b...,2019-01-10 20:19:18,2021-04-21 16:37:51,Here are this week's five links that are worth...,Web
3,thomas@collegeinfogeek.com,malvisbid@gmail.com,,,Build a complete productivity system - my firs...,2019-01-10 20:56:21,2021-04-21 16:09:42,Hey there!\nWe are now 10 full days into the n...,Computer
4,quincy@freecodecamp.org,malvisbid@gmail.com,,,How to build your own e-commerce website from ...,2019-01-17 20:45:13,2021-04-21 16:09:41,Here are this week's five links that are worth...,Web


In [27]:
import textwrap

def createPdf(efrom, to, cc, bcc, subject, date_sent, date_recieved, body, canvas):
    
    starting_y=800
    offset_y=0
    #pdf_file = subject
    #outfilepath = os.path.join( path, pdf_file )
    
    font_size= 11
    font_type_titles='Courier-Bold'
    font_type_normal= 'Courier'
    
    
    canvas.setFont(font_type_titles, font_size)
    canvas.drawString(30,starting_y,'Subject:')
    canvas.setFont(font_type_normal, font_size)
    canvas.drawString(stringWidth('Subject:', font_type_titles, font_size) +35,starting_y, subject)
    offset_y+=15

    canvas.setFont(font_type_titles, font_size)
    canvas.drawString(30,starting_y-offset_y,'From:')
    canvas.setFont(font_type_normal, font_size)
    canvas.drawString(stringWidth('From:', font_type_titles, font_size) +35,starting_y-offset_y,efrom)
    offset_y+=15

    canvas.setFont(font_type_titles, font_size)
    canvas.drawString(30,starting_y-offset_y,'CC:')
    canvas.setFont(font_type_normal, font_size)
    canvas.drawString( stringWidth('CC:', font_type_titles, font_size) +35,starting_y-offset_y,cc)
    offset_y+=15

    canvas.setFont(font_type_titles, font_size)
    canvas.drawString(30,starting_y-offset_y,'BCC:')
    canvas.setFont(font_type_normal, font_size)
    canvas.drawString(stringWidth('BCC:', font_type_titles, font_size) +35,starting_y-offset_y,bcc)
    offset_y+=15

    canvas.setFont(font_type_titles, font_size)
    canvas.drawString(30,starting_y-offset_y,'To:')
    canvas.setFont(font_type_normal, font_size)
    canvas.drawString(stringWidth('To:', font_type_titles, font_size) +35,starting_y-offset_y,to)
    offset_y+=15

    canvas.setFont(font_type_titles, font_size)
    canvas.drawString(30,starting_y-offset_y,'Date Sent:')
    canvas.setFont(font_type_normal, font_size)
    canvas.drawString(stringWidth('Data Sent:', font_type_titles, font_size) +35,starting_y-offset_y,date_sent)
    offset_y+=15

    canvas.setFont(font_type_titles, font_size)
    canvas.drawString(30,starting_y-offset_y,'Date Recieved:')
    canvas.setFont(font_type_normal, font_size)
    canvas.drawString(stringWidth('Date Recieved:', font_type_titles, font_size) +35,starting_y-offset_y,date_recieved)
    offset_y+=15

    canvas.setLineWidth(.7)
    canvas.line(30,700,580,700)
    
    canvas.setFont(font_type_normal, 10.5)
    coory=690-15
    if len(body) > 70:
        wrap_text = textwrap.wrap(body, width=85)
        for line in wrap_text:
            canvas.drawString(30, coory, line)
            coory-=15
    else:
        canvas.drawString(30, 690-15, body)
    

    canvas.save()

In [ ]:
#Create a directory
path_to_create_directory='C:/Users/HP/Downloads'

##before we start iterating over the dataframe, we have to group it according to Category
grouped = email_dataset_required.groupby('Labels')


for group_name, email_dataset_required_group in grouped:
    path=''
    new_dir= group_name
    path = os.path.join(path_to_create_directory, new_dir)
    try:
        os.mkdir(path)
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise
           
    columns_list=email_dataset_required_group.columns.to_list()
    for i in range(len(email_dataset_required_group)):
        createPdf(email_dataset_required_group.iloc[i, 0],email_dataset_required_group.iloc[i, 1],email_dataset_required_group.iloc[i, 2],email_dataset_required_group.iloc[i, 3],email_dataset_required_group.iloc[i, 4],
                  email_dataset_required_group.iloc[i, 5].strftime("%H:%M:%S.%f - %b %d %Y"),
                  email_dataset_required_group.iloc[i, 6].strftime("%H:%M:%S.%f - %b %d %Y") ,email_dataset_required_group.iloc[i, 7],
                  canvas.Canvas(os.path.join(path, re.sub('[^A-Za-z0-9]+', '', email_dataset_required_group.iloc[i, 4])+".pdf")))
                
    

In [ ]:


#newString=''
#data_words = list(sent_to_words(newString))
#removeStopwords=
#data_bigrammed=

#new_doc=newString 
#new_doc_bow = dictionary.doc2bow(new_doc)
#ldamodel.get_document_topics(new_doc_bow)

